In [1]:
# IMPLEMENT THE RESNET
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from typing import Type, Any, Callable, Union, List, Optional
import SimpleITK as sitk
import numpy as np
import os, sys, pathlib, math
import random
from skimage import data
import pywt

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tvtransforms
%matplotlib inline
import matplotlib.pyplot as plt

from torchviz import make_dot
import blocks, transforms, losses, ZhouModel, pix2pix, dataloaders, OhModel

data_BSE = os.path.normpath("D:/data/JSRT/augmented/augmented/target")
data_normal = os.path.normpath("D:/data/JSRT/augmented/augmented/source")
discriminator_keys_images = ["source","boneless"]
image_spatial_size = (256,256)
ds_discriminator = dataloaders.JSRT_CXR(data_normal, data_BSE, 
                                        transform=tvtransforms.Compose([
                             transforms.RescalingNormalisation(discriminator_keys_images,(0,1)),
                             transforms.Rescale(image_spatial_size, discriminator_keys_images, None),
                             transforms.HaarTransform(discriminator_keys_images),
                             transforms.ToTensor(discriminator_keys_images),
                             ]))



NameError: name 'image' is not defined

In [2]:
# DL
dl = DataLoader(ds_discriminator, batch_size=3, shuffle=True, num_workers=0)
sample = next(iter(dl))
image = sample["source"]
print("Image SHAPE is: " +str(image.shape))

# Minibatch Discrimination
mbd = blocks.MiniBatchDiscrimination(image.flatten(start_dim=1).shape[1], 50, 50)
print("Minibatch Discrimination")
print(mbd(image.flatten(start_dim=1)).shape)

# Losses
out = losses.criterion_TotalVariation(image)
print(out.shape)
out = losses.criterion_StyleReconstruction_layer(image, image, "mean")
print(out)

# Image Buffer
pool = blocks.ImageBuffer(10)
print(len(pool.images))
out = pool.query(image)
print(len(pool.images))
out = pool.query(image)
print(len(pool.images))

print("Testing UpsampleAndConv")
print("Image SHAPE is: " +str(image.shape))
blk = blocks.UpsampleConvolution(4,16,upsample_scale_factor=2,kernel_size=1)
print(blk(image).shape)

print("Oh Model")
blkC = OhModel.ConvResBlock(4,64)
print(blkC(image).shape)
blk = OhModel.DeconvResBlock(4,64)
print(blk(image).shape)
blkSE = OhModel.SqueezeExcitationBlock(in_c=4, out_c=16)
print(blkSE(image).shape)
Generator = OhModel.Generator(input_array_shape=image.shape)
#make_dot(Generator(image).mean(), params=dict(Generator.named_parameters()))

Disc = OhModel.Discriminator(image.shape, 50, 50)
print("Oh Model Discriminator")
print(Disc(image))


Image SHAPE is: torch.Size([3, 4, 128, 128])
Minibatch Discrimination
torch.Size([3, 65586])
torch.Size([3])
torch.Size([3, 4, 4])
tensor([0., 0., 0.])
0
3
6
Testing UpsampleAndConv
Image SHAPE is: torch.Size([3, 4, 128, 128])
torch.Size([3, 16, 256, 256])
Oh Model
torch.Size([3, 64, 64, 64])
torch.Size([3, 64, 256, 256])
torch.Size([3, 16, 128, 128])
Oh Model Generator thought to use summation skip connection.
Oh Model Discriminator
tensor([[-0.3090],
        [-0.0090],
        [ 0.0018]], grad_fn=<AddmmBackward>)


In [ ]:
a = torch.rand(2,3,4,5)
for b in a:
    print(b.shape)
print(3//2)

In [ ]:
STOP

# Test elementwise mult
a = torch.rand(2,1,4,4)
b = torch.zeros(2,1).unsqueeze(-1).unsqueeze(-1)
print((a*b))

# Pix2Pix
import torch
import torch.nn as nn
import pix2pix
from torchviz import make_dot
a = torch.rand(4,1,256,256)
gen = pix2pix.UnetGenerator(input_nc=1, output_nc=1, num_downs=8, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=True)
gen_out = gen(a)
make_dot(gen_out.mean(), params=dict(gen.named_parameters()))

In [ ]:
# Pix2Pix
import torch
import pix2pix
from torchviz import make_dot

a = torch.rand(4,1,256,256)
gen = pix2pix.Custom_Written_Generator(a.shape)
gen_out = gen(a)
make_dot(gen_out.mean(), params=dict(gen.named_parameters()))

In [ ]:
def set_requires_grad(nets, requires_grad=False):
    """https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/f13aab8148bd5f15b9eb47b690496df8dadbab0c/models/base_model.py#L219
    Set requires_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
        nets (network list)   -- a list of networks
        requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        if net is not None:
            for param in net.parameters():
                param.requires_grad = requires_grad
    return net

test = torch.randn(3)
model = nn.Sequential(nn.Linear(3,10),nn.Linear(10,10),nn.Linear(10,1))

model = set_requires_grad(model, False)
    
model = set_requires_grad(model, True)

In [ ]:
# TESTING PIX2PIX
from torchviz import make_dot, make_dot_from_trace
"""a = torch.randn(2,1,512,512)
b = torch.randn(2,1,1024,1024)

m = nn.ConvTranspose2d(1, 22, kernel_size=3, stride=2)
enc = blocks.Pix2Pix_Encoder_Block(1,22, _normType=None)
#print(enc(a).shape)
dec = blocks.Pix2Pix_DecoderBlock( _in_channels=1, _out_channels=22, _kernel_size=(4,4), _stride=(2,2), _padding=(1,1), _dilation=(1,1), _normType="BatchNorm", _dropoutType=None)
#print(dec(a,b).shape)
"""
# Generator
a = torch.randn(2,1,512,512)
generator = pix2pix.Generator_Pix2Pix(a.shape)
gen_out = generator(a)


# Discriminator
b = torch.randn(2,2,512,512)
disc = pix2pix.Discriminator_Pix2Pix(_input_array_size=b.shape, _first_out_channels=64, _normType="BatchNorm", spectral_normalize=False)
out = disc(b)

make_dot(gen_out.mean(), params=dict(generator.named_parameters()))

In [ ]:
# TESTING RESUNET (Zhang 2018)
def weights_init(m):
    # From DCGAN paper
    classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        if m.affine:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
    for i in m.children():
        # Specific weight setting for ResUNet shortcut.
        if i.__class__.__name__ == "ResUNet_shortcut":
            for ii in i.children():
                if isinstance(ii, nn.Conv2d) or isinstance(ii, nn.ConvTranspose2d):
                    nn.init.constant_(ii.weight.data, 1.)
            for param in i.parameters():
                param.requires_grad=False
                

a = torch.randn(2,1,512,512)
gen = blocks.Generator_ResUNet(input_array_shape=a.shape, _first_out_channels=64, _reluType="leaky")
gen.apply(weights_init)
print(gen(a).shape)


In [ ]:
a = torch.randn(2,1,512,512)
gen2 = blocks.Generator_ResUNet_PixelShuffle(input_array_shape=a.shape, _first_out_channels=64, _reluType="leaky")
out = gen2(a)
print(out[0,0,:,:])

In [ ]:
# TESTING RESUNET-A COMPONENTS

m = blocks.ResUNet_A_miniBlock(16)
n = blocks.Conv2DN(16,20)
p = blocks.ResUNet_A_Block_4(16, _kernel_size=(3,3), _dilation_rates=[1,3,5,7])
ds = blocks.DownSample(16)
mp = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2), padding=0, dilation=1, return_indices=False, ceil_mode=False)

a = torch.randn(1, 16, 256, 256)
c = torch.randn(1, 2, 512, 512)
d = torch.randn(1, 1024, 8, 8)

enc = blocks.Encoder_ResUNet_A_d7(32, c.shape)
msc = blocks.MultiScale_Classifier(_input_channels=32, _input_array_shape=c.shape)

print(m(a).shape)
print(n(a).shape)
print(p(a).shape)
print(ds(a).shape)
print(enc(c).shape)
print(msc(c).shape)

print("TESTING BRIDGE")

output_size = (a.shape[2]//4,a.shape[3]//4)
mp = blocks.PSPPooling_miniBlock(_in_channels=16, _output_size=output_size, _kernel_size=output_size, _stride=output_size, _padding=0, _dilation=(1,1), _pyramid_levels=4)
print(mp(a).shape)

pspp = blocks.PSPPooling(_tensor_array_shape = a.shape)
print(pspp(a).shape)


upsh = blocks.UpSampleAndHalveChannels( d.shape[1])
print(upsh(d).shape)

t1 = torch.randn(1,32,128,128)
upsh = blocks.UpSampleAndHalveChannels( t1.shape[1])
cbn= blocks.Combine(_in_channels=16)
print(cbn(a,upsh(t1)).shape)

In [ ]:
# Generator
print(tuple(map(lambda x: int(x), (1., 2.))))

input_test = torch.randn(1,2,512, 512)
generator = blocks.Generator_ResUNet_A(_input_channels=16, _input_array_shape=input_test.shape, _norm_type='BatchNorm', _ADL_drop_rate=0.75, _ADL_gamma=0.9)

print(generator(input_test).shape)